
This Notebook is about Applied Data Science Capstone Project ( week 3 ) 

In [3]:
import pandas as pd
import numpy as np


Week 3 from here

Importing Libraries

In [5]:
import lxml.html as lh
import requests
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab


print('Libraries imported.')

Libraries imported.


In [7]:
url='https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
page = requests.get(url)
doc = lh.fromstring(page.content)
tr_elements = doc.xpath('//tr')
tr_elements


[<Element tr at 0x7f3dd820b188>,
 <Element tr at 0x7f3dd820b318>,
 <Element tr at 0x7f3dd820b2c8>,
 <Element tr at 0x7f3dd820b408>,
 <Element tr at 0x7f3dd820b048>,
 <Element tr at 0x7f3dd820b0e8>,
 <Element tr at 0x7f3dd7b6c3b8>,
 <Element tr at 0x7f3dd7b6c408>,
 <Element tr at 0x7f3dd7b6c458>,
 <Element tr at 0x7f3dd7b6c4a8>,
 <Element tr at 0x7f3dd7b6c4f8>,
 <Element tr at 0x7f3dd7b6c548>,
 <Element tr at 0x7f3dd7b6c598>,
 <Element tr at 0x7f3dd7b6c7c8>,
 <Element tr at 0x7f3dd7b6c818>,
 <Element tr at 0x7f3dd7b6c868>,
 <Element tr at 0x7f3dd7b6c8b8>,
 <Element tr at 0x7f3dd7b6c908>,
 <Element tr at 0x7f3dd7b6c958>,
 <Element tr at 0x7f3dd7b6c9a8>,
 <Element tr at 0x7f3dd7b6c9f8>,
 <Element tr at 0x7f3dd7b6ca48>,
 <Element tr at 0x7f3dd7b6ca98>,
 <Element tr at 0x7f3dd7b6cae8>,
 <Element tr at 0x7f3dd7b6cb38>,
 <Element tr at 0x7f3dd7b6cb88>,
 <Element tr at 0x7f3dd7b6cbd8>,
 <Element tr at 0x7f3dd7b6cc28>,
 <Element tr at 0x7f3dd7b6cc78>,
 <Element tr at 0x7f3dd7b6ccc8>,
 <Element 

Getting columns names

In [9]:
tr_elements = doc.xpath('//tr')
#Create empty list
col=[]
i=0
#For each row, store each first element (header) and an empty list
for t in tr_elements[0]:
    i+=1
    name=t.text_content()
    print(i,name)
    col.append((name,[]))
    col

1 Postcode
2 Borough
3 Neighbourhood



Getting data

In [ ]:
for j in range(1,len(tr_elements)):
    #T is our j'th row
    T=tr_elements[j]
    
    #If row is not of size 3, the //tr data is not from our table 
    if len(T)!=3:
        break
    
    #i is the index of our column
    i=0
    
    #Iterate through each element of the row
    for t in T.iterchildren():
        data=t.text_content() 
        #Check if row is empty
        if i>0:
        #Convert any numerical value to integers
            try:
                data=int(data)
            except:
                pass
        #Append the data to the empty list of the i'th column
        col[i][1].append(data)
        #Increment i for the next column
        i+=1


Making DataFrame

In [12]:
Dict={title:column for (title,column) in col}
df=pd.DataFrame(Dict)

Removing records with Borough not assigned

In [13]:
df=df[df.Borough!='Not assigned']
df.head()

,Borough,Neighbourhood,Postcode
2,North York,Parkwoods\n,M3A
3,North York,Victoria Village\n,M4A
4,Downtown Toronto,Harbourfront\n,M5A
5,Downtown Toronto,Regent Park\n,M5A
6,North York,Lawrence Heights\n,M6A


Replacing Not assigned Neighbourhoods with Borough

In [14]:
indexes=df['Neighbourhood\n']=='Not assigned\n'
df.loc[indexes,'Neighbourhood\n']=df.loc[indexes,'Borough']
df.head()

,Borough,Neighbourhood,Postcode
2,North York,Parkwoods\n,M3A
3,North York,Victoria Village\n,M4A
4,Downtown Toronto,Harbourfront\n,M5A
5,Downtown Toronto,Regent Park\n,M5A
6,North York,Lawrence Heights\n,M6A


Combining Neighbourhoods

In [15]:
df = df.groupby(df['Postcode']).aggregate({'Borough': 'first', \
                                               'Neighbourhood\n': 'sum'})
df = df.reset_index()
df.head()

,Postcode,Borough,Neighbourhood
0,M1B,Scarborough,Rouge\nMalvern\n
1,M1C,Scarborough,Highland Creek\nRouge Hill\nPort Union\n
2,M1E,Scarborough,Guildwood\nMorningside\nWest Hill\n
3,M1G,Scarborough,Woburn\n
4,M1H,Scarborough,Cedarbrae\n


In [16]:
df.shape

(103, 3)